In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

from pymatgen.core import Structure, Lattice, Composition
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.ext.matproj import MPRester
from m3gnet.models import Relaxer



In [2]:
soup = BeautifulSoup(requests.get("https://en.wikipedia.org/wiki/Lattice_constant").text)

In [3]:
tbody = soup.find(class_="wikitable").find("tbody")
data = []
for tr in tbody.find_all("tr"):
    row = [td.text for td in tr.find_all('td')]
    if row:
        if row[2] not in ["Hexagonal", "Wurtzite", "Wurtzite (HCP)", "Orthorombic"]:
            try:
                a = float(row[1])
                data.append([row[0].split()[0].strip(), a, row[2].strip()])
            except ValueError:
                pass

data = pd.DataFrame(data, columns=["Formula", "a", "crystal"])
data = data.set_index("Formula")

In [4]:
print(data["crystal"].unique())
mpr = MPRester()

['Diamond (FCC)' 'Zinc blende (FCC)' 'Halite (FCC)' 'Halite'
 'Caesium chloride' 'FCC' 'BCC' 'Cubic perovskite']


In [5]:
predicted = []
mp = []
relaxation_time = []
relaxer = Relaxer()  # This loads the default pre-trained model

for formula, v in data.iterrows():
    
    c = Composition(formula)
    els = sorted(c.elements)
    if "Zinc blende" in v["crystal"]:
        s = Structure.from_spacegroup("F-43m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.25, 0.25, 0.75]])
    elif "Halite" in v["crystal"]:
        s = Structure.from_spacegroup("Fm-3m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.5, 0, 0]])
    elif "Caesium chloride" in v["crystal"]:
        s = Structure.from_spacegroup("Pm-3m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.5, 0.5, 0.5]])
    elif "Cubic perovskite" in v["crystal"]:
        s = Structure(Lattice.cubic(5), [els[0], els[1], els[2], els[2], els[2]], 
                      [[0., 0., 0.], [0.5, 0.5, 0.5], [0.5, 0.5, 0], [0., 0.5, 0.5], [0.5, 0, 0.5]])
    elif "Diamond" in v["crystal"]:
        s = Structure.from_spacegroup("Fd-3m", Lattice.cubic(5), [els[0]], [[0.25, 0.75, 0.25]])
    elif "BCC" in v["crystal"]:
        s = Structure(Lattice.cubic(5), [els[0]] * 2, [[0., 0., 0.], [0.5, 0.5, 0.5]])
    elif "FCC" in v["crystal"]:
        s = Structure(Lattice.cubic(5), [els[0]] * 4, [[0., 0., 0.], [0.5, 0.5, 0], [0., 0.5, 0.5], [0.5, 0, 0.5]])
    else:
        predicted.append(0)
        mp.append(0)
        continue
    
    
    start = datetime.datetime.now()

    relax_results = relaxer.relax(s)

    final_structure = relax_results['final_structure']

    timetaken = datetime.datetime.now() - start
     
    predicted.append(final_structure.lattice.a)
    relaxation_time.append(timetaken)
    
    try:
        entries = mpr.get_entries({"pretty_formula": s.composition.reduced_formula}, 
                                  inc_structure=True, property_data=["e_above_hull"])
        entries = sorted(entries, key=lambda e: e.data["e_above_hull"])
        for e in entries:
            try:
                sga = SpacegroupAnalyzer(e.structure)
                sga2 = SpacegroupAnalyzer(final_structure)
                if sga.get_space_group_number() == sga2.get_space_group_number():
                    conv = sga.get_conventional_standard_structure()
                    mp.append(conv.lattice.a)
                    break
            except:
                pass
        else:
            raise RuntimeError
    except Exception as ex:
        mp.append(0)
        import traceback
        traceback.print_exc()
data["predicted_a"] = predicted
data["mp_a"] = mp

/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("PartitionedCall:4", shape=(None,), dtype=int32), values=Tensor("PartitionedCall:3", shape=(None, 3, 3), dtype=float32), dense_shape=Tensor("PartitionedCall:5", shape=(3,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("PartitionedCall:1", shape=(544,), dtype=int32), values=Tensor("Neg:0", shape=(544, 3), dtype=float32), dense_shape=Tensor("PartitionedCall:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
2022-05-31 12:14:48.115558: W tensorflow/core/platform/profile_utils/cpu_

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 12:14:48      -36.148697*      38.4316


/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/m3g_net/three_d_interaction_2/GatherV2_1_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradients/m3g_net/three_d_interaction_2/GatherV2_1_grad/Reshape:0", dtype=float32), dense_shape=Tensor("gradients/m3g_net/three_d_interaction_2/GatherV2_1_grad/Cast:0", shape=(None,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/Users/shyue/miniconda3/envs/mavrl/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradients/m3g_net/graph_network_layer_2/gated_atom_update_2/GatherV2_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradients/m3g_net/graph_network_layer_2/gated_atom_update_2/GatherV2_grad/Reshape

FIRE:    1 12:14:50      -47.801781*      84.2779
FIRE:    2 12:14:50      -67.004982*      61.9953
FIRE:    3 12:14:50      -72.645134*       5.3897
FIRE:    4 12:14:50      -72.689499*       1.7229
FIRE:    5 12:14:50      -72.692436*       1.1855
FIRE:    6 12:14:50      -72.694916*       0.2681
FIRE:    7 12:14:50      -72.694031*       0.7506
FIRE:    8 12:14:50      -72.694176*       0.6899
FIRE:    9 12:14:50      -72.694435*       0.5736
FIRE:   10 12:14:50      -72.694733*       0.4114
FIRE:   11 12:14:50      -72.694962*       0.2167
FIRE:   12 12:14:50      -72.695045*       0.0054
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 12:14:52      -39.464790*      44.6512
FIRE:    1 12:14:52      -42.685207*       6.6691
FIRE:    2 12:14:52      -42.835457*       2.9842
FIRE:    3 12:14:52      -42.834694*       3.7586
FIRE:    4 12:14:52      -42.849213*       2.8557
FIRE:    5 12:14:52      -42.865707*       1.4411
FI

FIRE:    3 12:15:02      -27.507965*      12.9482
FIRE:    4 12:15:02      -28.111067*       6.6576
FIRE:    5 12:15:02      -28.207924*       5.8834
FIRE:    6 12:15:02      -28.243465*       4.4674
FIRE:    7 12:15:02      -28.284222*       2.3310
FIRE:    8 12:15:02      -28.302393*       0.1859
FIRE:    9 12:15:02      -28.290806*       1.6932
FIRE:   10 12:15:02      -28.291628*       1.6340
FIRE:   11 12:15:02      -28.293159*       1.5171
FIRE:   12 12:15:02      -28.295195*       1.3460
FIRE:   13 12:15:02      -28.297430*       1.1258
FIRE:   14 12:15:02      -28.299562*       0.8627
FIRE:   15 12:15:02      -28.301264*       0.5643
FIRE:   16 12:15:02      -28.302296*       0.2396
FIRE:   17 12:15:02      -28.302444*       0.1368
FIRE:   18 12:15:02      -28.302448*       0.1353
FIRE:   19 12:15:02      -28.302450*       0.1321
FIRE:   20 12:15:02      -28.302460*       0.1274
FIRE:   21 12:15:02      -28.302462*       0.1212
FIRE:   22 12:15:02      -28.302471*       0.1136


...

FIRE:   21 12:15:57      -74.567963*       0.4204
FIRE:   22 12:15:57      -74.568108*       0.3557
FIRE:   23 12:15:57      -74.568245*       0.2779
FIRE:   24 12:15:57      -74.568352*       0.1862
FIRE:   25 12:15:57      -74.568443*       0.0805
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 12:15:58      -74.961960*      51.7285
FIRE:    1 12:15:58      -80.698769*       0.0380


Traceback (most recent call last):
  File "/var/folders/ql/m5k56v8n5sz5880n5sksmc9w0000gn/T/ipykernel_35881/3188535382.py", line 57, in <cell line: 6>
    raise RuntimeError
RuntimeError


      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 12:15:59      -64.091125*      58.8516
FIRE:    1 12:15:59      -74.576164*      16.9512
FIRE:    2 12:15:59      -66.967651*     113.8145
FIRE:    3 12:15:59      -74.576164*      16.9512
FIRE:    4 12:15:59      -74.844917*      10.2579
FIRE:    5 12:15:59      -75.006470*       0.2538
FIRE:    6 12:15:59      -75.006493*       0.2278
FIRE:    7 12:15:59      -75.006516*       0.1785
FIRE:    8 12:15:59      -75.006554*       0.1110
FIRE:    9 12:15:59      -75.006554*       0.0323
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 12:16:00      -75.339874*      34.3569
FIRE:    1 12:16:00      -73.843277*      69.1700
FIRE:    2 12:16:00      -75.339874*      34.3569
FIRE:    3 12:16:00      -76.470688*      22.0703
FIRE:    4 12:16:00      -77.235336*       0.1467
FIRE:    5 12:16:00      -77.235352*       0.1290
FIRE

Traceback (most recent call last):
  File "/var/folders/ql/m5k56v8n5sz5880n5sksmc9w0000gn/T/ipykernel_35881/3188535382.py", line 57, in <cell line: 6>
    raise RuntimeError
RuntimeError


FIRE:    7 12:16:05      -88.796356*       0.4686
FIRE:    8 12:16:05      -88.796608*       0.0803
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 12:16:05      -68.883659*      45.4988
FIRE:    1 12:16:05      -73.877228*       9.2434
FIRE:    2 12:16:05      -73.952057*       5.5051
FIRE:    3 12:16:05      -73.980797*       3.4100
FIRE:    4 12:16:05      -73.998398*       0.0513
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 12:16:06      -31.849533*      32.1436
FIRE:    1 12:16:06      -38.329632*      25.3043
FIRE:    2 12:16:06      -40.979641*       6.3708
FIRE:    3 12:16:06      -41.073353*       0.0311
      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 12:16:07      -30.478622*      29.8102
FIRE:    1 12:16:07      -36.794582*      24.8096
FIRE:    2 12:16:07      -40.091175*       1.9120
FIRE: 

In [6]:
data["% error vs exp"] = (data["predicted_a"] - data["a"])/data["a"] * 100
data["% error vs mp"] = (data["predicted_a"] - data["mp_a"])/data["mp_a"] * 100

In [7]:
print(data.to_markdown())

| Formula   |       a | crystal           |   predicted_a |    mp_a |   % error vs exp |   % error vs mp |
|:----------|--------:|:------------------|--------------:|--------:|-----------------:|----------------:|
| C         | 3.567   | Diamond (FCC)     |       3.5718  | 3.57371 |        0.13468   |     -0.0533331  |
| Si        | 5.43102 | Diamond (FCC)     |       5.45002 | 5.46873 |        0.349891  |     -0.342032   |
| Ge        | 5.658   | Diamond (FCC)     |       5.7698  | 5.76286 |        1.97602   |      0.120446   |
| AlAs      | 5.6605  | Zinc blende (FCC) |       5.73027 | 5.73376 |        1.23259   |     -0.0607868  |
| AlP       | 5.451   | Zinc blende (FCC) |       5.50346 | 5.50711 |        0.962427  |     -0.0662799  |
| AlSb      | 6.1355  | Zinc blende (FCC) |       6.22817 | 6.23376 |        1.51036   |     -0.0897066  |
| GaP       | 5.4505  | Zinc blende (FCC) |       5.5054  | 5.5063  |        1.00718   |     -0.0164147  |
| GaAs      | 5.653   | Zinc blende (